# Scrape PMIDs from Genbank entries

June 22, 2018 

I think that one of the challenges of doing the H5N1 project that I'm proposing is that in the databases (both IRD and GISAID) generally there is not any annotation of which sequences come from wild vs. domestic birds. This is primarily an issue for ducks and geese, which are widely farmed but also are wild. It seems like the only way to actually get this information is to follow the genbank links and use them to get the pubmed IDs, and then read the paper. When you do this, it is often pretty obvious which samples are wild vs. domestic. However, this would reqiure manually annotating thousands of sequences. In order to get a grasp on how difficult this would be, I would like to take an input of entries with genbank numbers, follow those links to get the pubmed ID, and then make a list of unique pubmed IDs. That would give me a better sense for how many papers would actually need to be read. I am currently using a list from GISAID including all H5N1 sequences from duck, geese, or "other avian" species. 

In [9]:
# using the Entrez feature from bio
from Bio import Entrez
from Bio import SeqIO
import re

In [10]:
from itertools import islice
def take(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))


In [11]:
# read in csv file and make a list of accession numbers to query

def get_accession_numbers(csv_file):
    accession_numbers = {}

    with open(input_csv, "r") as infile:
        for line in infile:
            if "Isolate_Id" not in line:

                domestic_wild = line.split("\t")[11]
                
                genbank_accessions = [line.split("\t")[15],line.split("\t")[16],line.split("\t")[17],line.split("\t")[18],line.split("\t")[19],line.split("\t")[20],line.split("\t")[21],line.split("\t")[22]]
                new_accessions = []
                to_remove = []
                
                for g in genbank_accessions:
                    
                    # this is necessary to take care of instances when there are 2 genbank IDs for a single entry, 
                    # which will show up like "AB450517, EF178510" 
                    if len(g.split(",")) > 1:
                        
                        for i in range(0,len(g.split(","))):
                            new_accessions.append(g.split(",")[i].replace("\"","").replace(" ",""))
                
                        to_remove.append(g)
                
                for n in new_accessions:
                    genbank_accessions.append(n)
                for t in to_remove:
                    genbank_accessions.remove(t)
                    
                for g in genbank_accessions:    
                    if g != "":
                        accession_numbers[g] = domestic_wild
                
    return(accession_numbers)

Fetch a record as a genbank file

In [12]:
def fetch_genbank_entry(accession_number):

    Entrez.email = "lhmoncla@gmail.com"      # need to enter an email

    # fetch from the nucleotide database, return a genbank object in text format, with id = genbank accession
    handle = Entrez.efetch(db="nucleotide", rettype="gb", retmode="text", id=accession_number)
    genbank_result = handle.read()
    return(genbank_result)

In [13]:
def return_PMID(genbank_result):
    SearchStr = 'PUBMED\\ \\ \\ [0-9]{8}'
    result = re.search(SearchStr,genbank_result)

    if result:
        PMID = result.group(0).split("   ")[1]
        return(PMID)

### Read input csv and feed into functions

I am using a csv file that I downloaded from GISAID of all H5N1 avian sequences labelled as duck, goose, or other avian. 

In [203]:
input_csv = "/Users/lmoncla/Documents/Fellowships/data_query/gisaid_epiflu_isolates_h5n1_goose_duck_otheravian_withannotations2.csv"
PMIDs = {}

In [204]:
# run everything and write out the pubmed IDs to a dictionary

accession_numbers = get_accession_numbers(input_csv)      # get accession numbers from csv

print("finding pubmed IDs...")

for a in accession_numbers:
    if len(a.split(","))>1:
        print(a)
    #print(a)
    domestic_wild = accession_numbers[a]
    genbank_result = fetch_genbank_entry(a)       # get the genbank result
    PMID = return_PMID(genbank_result)            # search for a PMID and return it 
    
    if PMID != None and PMID not in PMIDs:        # write pubmed ID, genbank number, and domestic/wild to dict
        PMIDs[PMID] = {}
        PMIDs[PMID][a] = domestic_wild
    elif PMID != None and PMID in PMIDs:
        PMIDs[PMID][a] = domestic_wild

finding pubmed IDs...


In [205]:
print(len(PMIDs))

167


In [217]:
# write this out to a text file, so that if I accidentally start running the above again, I don't have to wait for 
# it to finish
outfilename = "/Users/lmoncla/Documents/Fellowships/data_query/pubmed_ids_2018-06-28.txt"
with open(outfilename, "w") as outfile:
    for p in PMIDs:
        for g in PMIDs[p]:
            outfile.write(p + "\t" + g + "\t" + PMIDs[p][g] + "\n")

In [270]:
# print out the title, journal, authors, and abstract for articles based on the pubmed ID. 
for p in PMIDs:
    pmid_handle = Entrez.efetch(db='pubmed', id=p, retmode='text', rettype='abstract')
    print(p)
    print(pmid_handle.read())

23211427

1. Vet Microbiol. 2013 Mar 23;162(2-4):471-8. doi: 10.1016/j.vetmic.2012.10.034.
Epub 2012 Nov 7.

Characterization of mutations associated with the adaptation of a low-pathogenic 
H5N1 avian influenza virus to chicken embryos.

Kim IH(1), Kwon HJ, Choi JG, Kang HM, Lee YJ, Kim JH.

Author information: 
(1)Laboratory of Avian Diseases, Seoul National University, Seoul 151-742,
Republic of Korea.

Migratory waterfowls are the most common reservoir for avian influenza virus
(AIV), thus viral adaptation is required for efficient replication in land fowls.
To date, low pathogenic (LP) H5 subtype AIVs have been isolated from migratory
waterfowls, and the adaptation of these viruses to land fowls might lead to the
generation of highly pathogenic AIVs. Thus, A/wild duck/Korea/50-5/2009 (H5N1)
LPAIV was passaged 20 times through embryonated chicken eggs (ECEs), and the
resulting genetic and phenotypic changes were investigated. The pathogenicities
of the early (50-5-E2) and final pas

26627108

1. Sci Rep. 2015 Dec 2;5:17508. doi: 10.1038/srep17508.

Genetic and molecular characterization of H9N2 and H5 avian influenza viruses
from live poultry markets in Zhejiang Province, eastern China.

Wu H(1), Peng X(1), Peng X(1), Cheng L(1), Lu X(1), Jin C(1), Xie T(1), Yao H(1),
Wu N(1).

Author information: 
(1)State Key Laboratory for Diagnosis and Treatment of Infectious Diseases,
Collaborative Innovation Center for Diagnosis and Treatment of Infectious
Diseases, the First Affiliated Hospital, School of Medicine, Zhejiang University,
310003 Hangzhou, China.

Live poultry markets (LPMs) are a key source of reassorted avian influenza
viruses (AIVs) because of the density of terrestrial and aquatic poultry and the 
frequency of AIV infection. H9N2 viruses are prevalent in terrestrial poultry
throughout Asia and have been isolated from poultry outbreaks worldwide. They
infect both avian and mammalian species and may be significant donors of genetic 
material to emerging human

19520524

1. Vet Microbiol. 2009 Oct 20;139(1-2):24-33. doi: 10.1016/j.vetmic.2009.04.017.
Epub 2009 Apr 19.

Four different sublineages of highly pathogenic avian influenza H5N1 introduced
in Hungary in 2006-2007.

Szeleczky Z(1), Dán A, Ursu K, Ivanics E, Kiss I, Erdélyi K, Belák S, Muller CP, 
Brown IH, Bálint A.

Author information: 
(1)Department of Virology, Central Agricultural Office Veterinary Diagnostic
Directorate, Budapest, Hungary.

Highly pathogenic avian influenza (HPAI) H5N1 viruses were introduced to Hungary 
during 2006-2007 in three separate waves. This study aimed at determining the
full-length genomic coding regions of the index strains from these epizootics in 
order to: (i) understand the phylogenetic relationship to other European H5N1
isolates, (ii) elucidate the possible connection between the different outbreaks 
and (iii) determine the putative origin and way of introduction of the different 
virus variants. Molecular analysis of the HA gene of Hungarian HPA

18172752

1. Virus Genes. 2008 Feb;36(1):117-25. doi: 10.1007/s11262-007-0188-7. Epub 2008 Jan
3.

Genetic characterization of the NS gene indicates co-circulation of two
sub-lineages of highly pathogenic avian influenza virus of H5N1 subtype in
Northern Europe in 2006.

Zohari S(1), Gyarmati P, Thorén P, Czifra G, Bröjer C, Belák S, Berg M.

Author information: 
(1)Department of Virology, Joint Research and Development in Virology, National
Veterinary Institute (SVA), Ulls väg 2B, 751 89 Uppsala, Sweden
siamak.zohari@sva.se

The non-structural (NS) gene of highly pathogenic avian influenza viruses of the 
H5N1 subtype (HPAI-H5N1) isolated in Baltic Sea area of Sweden in 2006 was
studied. The phylogenetic analysis data demonstrated that two distinct
sub-lineages of HPAI-H5N1 were circulating during the outbreak in Northern Europe
in Spring 2006. Sub-lineage I viruses fell into the same clade as viruses found
in Denmark and Germany and formed a sub-clade which also included viruses
isol

16532371

1. Arch Virol. 2006 Aug;151(8):1615-24. Epub 2006 Mar 13.

Detection of Hong Kong 97-like H5N1 influenza viruses from eggs of Vietnamese
waterfowl.

Li Y(1), Lin Z, Shi J, Qi Q, Deng G, Li Z, Wang X, Tian G, Chen H.

Author information: 
(1)Animal Influenza Laboratory of the Ministry of Agriculture Sciences and
National Key Laboratory of Veterinary Biotechnology, Harbin Veterinary Research
Institute, CAAS, Harbin, PR China.

Three H5N1 influenza viruses were isolated from shell washes of duck and goose
eggs confiscated from travelers coming from Vietnam. All eight gene segments of
these viruses share high sequence identity with the H5N1 avian influenza viruses 
that caused outbreaks in poultry and humans in Hong Kong in 1997. Animal studies 
indicate that these isolated viruses are able to replicate in mouse lung and
could be found in the organs of ducks without causing any clinical signs or
death. However, the viruses are highly pathogenic for chickens. Although the
source o

...

17694626

1. Vet Rec. 2007 Jul 21;161(3):100-1.

Outbreak of highly pathogenic avian influenza caused by Asian lineage H5N1 virus 
in turkeys in Great Britain in January 2007.

Irvine RM(1), Banks J, Londt BZ, Lister SA, Manvell RJ, Outtrim L, Russell C, Cox
WJ, Ceeraz V, Shell W, Landeg FJ, Wilesmith JW, Alexander DJ, Brown IH.

Author information: 
(1)Virology Department, VLA-Weybridge, Surrey KT15 3NB.


PMID: 17694626  [Indexed for MEDLINE]


20580497

1. Vet Microbiol. 2011 Jan 10;147(1-2):170-5. doi: 10.1016/j.vetmic.2010.05.045.
Epub 2010 Jun 9.

Genetic analyses of H5N1 avian influenza virus in Mongolia, 2009 and its
relationship with those of eastern Asia.

Kang HM(1), Batchuluun D, Kim MC, Choi JG, Erdene-Ochir TO, Paek MR, Sugir T,
Sodnomdarjaa R, Kwon JH, Lee YJ.

Author information: 
(1)Avian Disease Division, National Veterinary Research and Quarantine Service,
173, Jungangro, Manangu, Anyangsi, Gyeonggido 430-757, Republic of Korea.

Erratum in
    Vet Microbiol. 2011 Ma

18564674

1. Southeast Asian J Trop Med Public Health. 2008 May;39(3):373-82.

Whole genome sequences of H5N1 influenza a virus isolated from a little grebe in 
Thailand.

Naksupan N(1), Sanguansermsri D, Wongvilairat R, Niumsup PR, Pongcharoen S,
Chamnanpood P, Chamnanpood C, Sanguansermsri P.

Author information: 
(1)Faculty of Medical Sciences, Naresuan University, Phitsanulok, Thailand.

This is the first report of the whole genome sequence of influenza A virus in an 
aquatic resident bird of Thailand. It was categorized into genotype Z according
to its characteristics of a 20 amino acid deletion in neuraminidase and a five
amino acid deletion in the nonstructural protein. The indicator for a highly
pathogenic trait of the virus is the presence of a polybasic amino acid sequence 
at the cleavage site of HA0. The feature of resistance to the antiviral drug
amantadine is found at the 31st amino acid position of M2 (serine to asparagine).
Phylogenic analyses revealed that virus A/litt

24211664

1. Virus Res. 2014 Jan 22;179:125-32. doi: 10.1016/j.virusres.2013.10.023. Epub 2013
Nov 5.

Pathogenicity of an H5N1 avian influenza virus isolated in Vietnam in 2012 and
reliability of conjunctival samples for diagnosis of infection.

Bui VN(1), Dao TD(2), Nguyen TT(2), Nguyen LT(2), Bui AN(2), Trinh DQ(1), Pham
NT(2), Inui K(3), Runstadler J(4), Ogawa H(5), Nguyen KV(2), Imai K(6).

Author information: 
(1)Obihiro University of Agriculture and Veterinary Medicine, 2-11 Inada,
Obihiro, Hokkaido 080 8555, Japan; National Institute of Veterinary Research, 86 
Truong Chinh, Dong Da, Hanoi, Viet Nam.
(2)National Institute of Veterinary Research, 86 Truong Chinh, Dong Da, Hanoi,
Viet Nam.
(3)Food and Agriculture Organization, 3 Nguyen Gia Thieu, Hoan Kiem, Hanoi, Viet 
Nam.
(4)Massachusetts Institute of Technology, Cambridge, MA 02139, USA.
(5)Obihiro University of Agriculture and Veterinary Medicine, 2-11 Inada,
Obihiro, Hokkaido 080 8555, Japan. Electronic address: hogawa@obih

20836384

1. Bing Du Xue Bao. 2010 Jul;26(4):298-304.

[Genome sequencing and genetic analysis of a natural reassortant H5N1 subtype
avian influenza virus possessing H9N2 internal genes].

[Article in Chinese]

Gu M(1), Liu WB, Cao JP, Cao YZ, Zhang XR, Peng DX, Liu XF.

Author information: 
(1)Key Laboratory of Animal Infectious Diseases, Yang Zhou University, Yang Zhou 
225009, China. lucygu083@yahoo.com.cn

Abstract:One H5N1 subtype avian influenza virus, A/duck/Shandong/009/2008
(Dk/SD/009/08), was isolated from apparently healthy domestic ducks in some live 
bird market in East China during our epidemiological surveillance. To investigate
the genetic composition, Dk/SD/009/08 was subjected to genome sequencing. The
amino acid motif of cleavage site was "PLRERRRK-R/GL", which was consistent with 
the characterization of the HPAIV. According to the newest unified nomenclature
system of H5N1, Dk/SD/ 009/08 was classified into Clade 2.3.4. The BLAST results 
showed that four gene segm

16439620

1. Science. 2006 Mar 17;311(5767):1576-80. Epub 2006 Jan 26.

Large-scale sequence analysis of avian influenza isolates.

Obenauer JC(1), Denson J, Mehta PK, Su X, Mukatira S, Finkelstein DB, Xu X, Wang 
J, Ma J, Fan Y, Rakestraw KM, Webster RG, Hoffmann E, Krauss S, Zheng J, Zhang Z,
Naeve CW.

Author information: 
(1)Hartwell Center for Bioinformatics and Biotechnology, St. Jude Children's
Research Hospital, Memphis, TN 38105, USA.

Comment in
    Science. 2006 Mar 17;311(5767):1562-3.
    Science. 2006 Sep 15;313(5793):1573; author reply 1573.

The spread of H5N1 avian influenza viruses (AIVs) from China to Europe has raised
global concern about their potential to infect humans and cause a pandemic. In
spite of their substantial threat to human health, remarkably little AIV
whole-genome information is available. We report here a preliminary analysis of
the first large-scale sequencing of AIVs, including 2196 AIV genes and 169
complete genomes. We combine this new informati

### Read in text file with pubmed IDs and the annotation on them 

In [1]:
annotations = "/Users/lmoncla/Documents/Fellowships/data_query/domestic_wild_annotations_PMIDs_2018-06-25.txt"
PMID_dict_file = "/Users/lmoncla/Documents/Fellowships/data_query/pubmed_ids_2018-06-28.txt"

In [2]:
# read in PMID_dict_file and use to generate PMID dictinary in case I am reloading this 

PMIDs = {}

with open(PMID_dict_file, "r") as infile: 
    for line in infile: 
        pubmedid = line.split("\t")[0].lower()
        genbank_accession = line.split("\t")[1].lower()
        domestic_wild = line.split("\t")[2].strip().lower()
        
        if pubmedid not in PMIDs:
            PMIDs[pubmedid] = {}
        
        if pubmedid in PMIDs and genbank_accession not in PMIDs[pubmedid]:
            PMIDs[pubmedid][genbank_accession] = domestic_wild
            
            
print(PMIDs)

{'23211427': {'jx497773': 'wild', 'jx497774': 'wild', 'jx497775': 'wild', 'jx497776': 'wild', 'jx497777': 'wild', 'jx497778': 'wild', 'jx497779': 'wild', 'jx497772': 'wild', 'jx497780': 'wild', 'jx497765': 'wild', 'jx497766': 'wild', 'jx497767': 'wild', 'jx497768': 'wild', 'jx497769': 'wild', 'jx497770': 'wild', 'jx497771': 'wild'}, '19026689': {'eu057725': 'wild'}, '21673160': {'jf510040': 'wild', 'jf510045': 'wild', 'jf510046': 'wild', 'jf510041': 'wild', 'jf510043': 'wild', 'jf510047': 'wild', 'jf510042': 'wild', 'jf510044': 'wild'}, '25838478': {'kp739418': 'wild', 'kp739419': 'wild', 'kp739420': 'wild', 'kp739421': 'wild', 'kp739422': 'wild', 'kp739423': 'wild', 'kp739424': 'wild', 'kp739425': 'wild'}, '23950136': {'kc690153': 'domestic', 'kc690154': 'domestic', 'kc690155': 'domestic', 'kc690156': 'domestic', 'kc690157': 'domestic', 'kc690158': 'domestic', 'kc690159': 'domestic', 'kc690160': 'domestic'}, '23861018': {'ab780475': 'domestic', 'ab780476': 'domestic', 'ab780477': 'dom

In [3]:
test_PMID_dict = PMIDs

with open(annotations, "r") as infile:
    for line in infile:
        if "PMID" not in line: 
            PMID = line.split("\t")[0]
            annotation = line.split("\t")[1].replace("\n","")
            
            if PMID in test_PMID_dict and "requires reading the paper" not in annotation and "exclude" not in annotation:
                for t in test_PMID_dict[PMID]:
                    test_PMID_dict[PMID][t] = annotation
                    
print(test_PMID_dict)

{'23211427': {'jx497773': 'wild', 'jx497774': 'wild', 'jx497775': 'wild', 'jx497776': 'wild', 'jx497777': 'wild', 'jx497778': 'wild', 'jx497779': 'wild', 'jx497772': 'wild', 'jx497780': 'wild', 'jx497765': 'wild', 'jx497766': 'wild', 'jx497767': 'wild', 'jx497768': 'wild', 'jx497769': 'wild', 'jx497770': 'wild', 'jx497771': 'wild'}, '19026689': {'eu057725': 'wild'}, '21673160': {'jf510040': 'wild', 'jf510045': 'wild', 'jf510046': 'wild', 'jf510041': 'wild', 'jf510043': 'wild', 'jf510047': 'wild', 'jf510042': 'wild', 'jf510044': 'wild'}, '25838478': {'kp739418': 'wild', 'kp739419': 'wild', 'kp739420': 'wild', 'kp739421': 'wild', 'kp739422': 'wild', 'kp739423': 'wild', 'kp739424': 'wild', 'kp739425': 'wild'}, '23950136': {'kc690153': 'domestic', 'kc690154': 'domestic', 'kc690155': 'domestic', 'kc690156': 'domestic', 'kc690157': 'domestic', 'kc690158': 'domestic', 'kc690159': 'domestic', 'kc690160': 'domestic'}, '23861018': {'ab780475': 'domestic', 'ab780476': 'domestic', 'ab780477': 'dom

In [4]:
count = 0
wild = 0
domestic = 0

for t in test_PMID_dict:
    for i in test_PMID_dict[t]:
        if test_PMID_dict[t][i] == "":
            count += 1
        elif test_PMID_dict[t][i] == "wild":
            wild += 1
        elif test_PMID_dict[t][i] == "domestic":
            domestic +=1 
            
print(count, wild, domestic)

2404 604 1995


In [14]:
# check how many of my sequences are actually in my pubmed IDs dictionary

alignment = "/Users/lmoncla/Documents/H5N1_Cambodian_outbreak_study/beast/dta-2018-08-23/HA_2018-08-08-v2.aligned.reordered-short-removed.fasta"

PMIDs2 = {}

# make another dictionary, PMIDs2 that has just genbank_number : domestic/wild_status
for p in test_PMID_dict:
    for g in test_PMID_dict[p]:
        PMIDs2[g] = test_PMID_dict[p][g]

yes = 0 
no = 0
not_available = 0
no_list = []

with open(alignment, "r") as infile: 
    for line in infile:
        if ">" in line: 
            strain = line.split("|")[0]
            if len(strain.split("/")) == 4:
                species = "human"
            elif len(strain.split("/")) < 4:
                species = "unknown"
            else:
                species = strain.split("/")[1]
                
            genbank = line.split("|")[6]
            
            if species == "duck" or species == "goose":
                if genbank in PMIDs2:
                    #print("yes")
                    yes+= 1
                else:
                    if genbank == 'not_available' or genbank == 'unknown':
                    #print("no", genbank)
                        not_available += 1
                    
                    else:
                        no += 1
                        no_list.append(genbank)
                    
print(yes, no, not_available)

52 138 120


In [15]:
print(no_list)

['cy126096', 'ab593447', 'kf715205', 'gu182166', 'ab824279', 'cy058815', 'kf001434', 'kp097892', 'kf258189', 'kr732408', 'kp097891', 'kp288324', 'kc261464', 'ku143259', 'ku143260', 'kp097898', 'gu252830', 'gu252822', 'cy044040', 'ky171438', 'kx513126', 'kx513181', 'kx513230', 'ky171736', 'ky171470', 'kx513166', 'ab972699', 'ab819037', 'ky415618', 'mg042465', 'ky635769', 'ky171552', 'ky171183', 'mg042257', 'ab979471', 'kx513264', 'ky171454', 'lc363996', 'ky171191', 'kp336312', 'lc364273', 'ky171560', 'ky635638', 'lc167526', 'lc363980', 'ky171576', 'lc167510', 'ky171430', 'ab972731', 'kx513187', 'ky171624', 'mg042312', 'kt362022', 'ky171608', 'ab742295', 'ky171406', 'ky171111', 'mg042204', 'mg042457', 'ky171366', 'ab979503', 'ky170876', 'ky171600', 'ky171175', 'mg042099', 'mg042432', 'ky171568', 'mg042329', 'ky171207', 'ab742271', 'mg042277', 'kx513365', 'ky171414', 'lc167590', 'ab972707', 'kx513336', 'kt361988', 'kf001461', 'lc363988', 'ky171422', 'ky635518', 'lc364004', 'mg042382', 'ky

In [16]:
# go through this genbank accessions list and search for the pubmed IDs
extra_PMIDs = []

for i in no_list:
    print(i)
    genbank_result = fetch_genbank_entry(i)       # get the genbank result
    print(genbank_result)
    PMID = return_PMID(genbank_result)            # search for a PMID and return it 
    extra_PMIDs.append(PMID)

cy126096
LOCUS       CY126096                1749 bp    cRNA    linear   VRL 04-FEB-2013
DEFINITION  Influenza A virus (A/duck/El Fayoum/CAI1/2010(H5N1)) hemagglutinin
            (HA) gene, complete cds.
ACCESSION   CY126096
VERSION     CY126096.1
KEYWORDS    .
SOURCE      Influenza A virus (A/duck/El Fayoum/CAI1/2010(H5N1))
  ORGANISM  Influenza A virus (A/duck/El Fayoum/CAI1/2010(H5N1))
            Viruses; ssRNA viruses; ssRNA negative-strand viruses;
            Orthomyxoviridae; Influenzavirus A.
REFERENCE   1  (bases 1 to 1749)
  AUTHORS   Wentworth,D.E., Dugan,V., Halpin,R., Lin,X., Bera,J., Ghedin,E.,
            Fedorova,N., Overton,L., Tsitrin,T., Stockwell,T., Amedeo,P.,
            Bishop,B., Chen,H., Edworthy,P., Gupta,N., Katzel,D., Li,K.,
            Schobel,S., Shrivastava,S., Thovarai,V., Wang,S., Atta
            El-Abiary,E., Saad El-Din Abd El-Latif,W., Ahmed Khder,A., Selim
            Salama,S., Aly,M., El-Satar Arafa Mohamed Arafa,A., Bao,Y.,
            Sanders

LOCUS       KF715205                1742 bp    cRNA    linear   VRL 23-DEC-2013
DEFINITION  Influenza A virus (A/duck/Bangladesh/19097/2013(H5N1)) segment 4
            hemagglutinin (HA) gene, complete cds.
ACCESSION   KF715205
VERSION     KF715205.1
KEYWORDS    .
SOURCE      Influenza A virus (A/duck/Bangladesh/19097/2013(H5N1))
  ORGANISM  Influenza A virus (A/duck/Bangladesh/19097/2013(H5N1))
            Viruses; ssRNA viruses; ssRNA negative-strand viruses;
            Orthomyxoviridae; Influenzavirus A.
REFERENCE   1  (bases 1 to 1742)
  AUTHORS   Feeroz,M.M., Alam,R.S., Hasan,K.M., McKenzie,P.,
            Marinova-Petkova,A. and Webster,R.G.
  TITLE     Multiple introductions of highly pathogenic avian influenza H5N1
            viruses into Bangladesh
  JOURNAL   Unpublished
REFERENCE   2  (bases 1 to 1742)
  AUTHORS   Feeroz,M.M., Alam,R.S., Hasan,K.M., McKenzie,P.,
            Marinova-Petkova,A. and Webster,R.G.
  TITLE     Direct Submission
  JOURNAL   Submitted (11-OCT-20

LOCUS       AB824279                1742 bp    cRNA    linear   VRL 15-JUN-2013
DEFINITION  Influenza A virus (A/duck/Quanh Ninh/21/2013(H5N1)) viral cRNA,
            segment 4, complete sequence.
ACCESSION   AB824279
VERSION     AB824279.1
KEYWORDS    .
SOURCE      Influenza A virus (A/duck/Quang Ninh/21/2013(H5N1))
  ORGANISM  Influenza A virus (A/duck/Quang Ninh/21/2013(H5N1))
            Viruses; ssRNA viruses; ssRNA negative-strand viruses;
            Orthomyxoviridae; Influenzavirus A.
REFERENCE   1
  AUTHORS   Azuma,N., Soda,K., Hotta,K., Le,M.Q. and Ito,T.
  TITLE     Avian influenza virus surveillance in poultry farms and wet markets
            in Vietnam, from Sep 2011 to Mar 2014
  JOURNAL   Unpublished
REFERENCE   2  (bases 1 to 1742)
  AUTHORS   Azuma,N., Soda,K. and Ito,T.
  TITLE     Direct Submission
  JOURNAL   Submitted (04-JUN-2013) Contact:Noriko Azuma Tottori University,
            Avian Zoonosis Research Center,Faculty of Agriculture; 4-101
            Koyama-

LOCUS       KF001434                1713 bp    cRNA    linear   VRL 01-OCT-2014
DEFINITION  Influenza A virus (A/duck/Cambodia/143W4M4/2013(H5N1)) segment 4
            hemagglutinin (HA) gene, complete cds.
ACCESSION   KF001434
VERSION     KF001434.1
KEYWORDS    .
SOURCE      Influenza A virus (A/duck/Cambodia/143W4M4/2013(H5N1))
  ORGANISM  Influenza A virus (A/duck/Cambodia/143W4M4/2013(H5N1))
            Viruses; ssRNA viruses; ssRNA negative-strand viruses;
            Orthomyxoviridae; Influenzavirus A.
REFERENCE   1  (bases 1 to 1713)
  AUTHORS   Buchy,P.
  TITLE     Internal genes reassortment and antigenic drift of Influenza
            A(H5N1) in Cambodia
  JOURNAL   Unpublished
REFERENCE   2  (bases 1 to 1713)
  AUTHORS   Buchy,P.
  TITLE     Direct Submission
  JOURNAL   Submitted (30-APR-2013) Virology, Instiutut Pasteur in Cambodia, 5,
            Monivong Boulevards, PB 983, Phnom Penh 12201, Cambodia
FEATURES             Location/Qualifiers
     source          1..1713


LOCUS       KF258189                1722 bp    cRNA    linear   VRL 19-DEC-2013
DEFINITION  Influenza A virus (A/duck/Egypt/S6419C/2012(H5N1)) segment 4
            hemagglutinin (HA) gene, complete cds.
ACCESSION   KF258189
VERSION     KF258189.1
KEYWORDS    .
SOURCE      Influenza A virus (A/duck/Egypt/S6419C/2012(H5N1))
  ORGANISM  Influenza A virus (A/duck/Egypt/S6419C/2012(H5N1))
            Viruses; ssRNA viruses; ssRNA negative-strand viruses;
            Orthomyxoviridae; Influenzavirus A.
REFERENCE   1  (bases 1 to 1722)
  AUTHORS   Kayali,G., El-Shesheny,R., Kandeil,A.M., Webby,R.J. and Ali,M.A.
  TITLE     Direct Submission
  JOURNAL   Submitted (17-JUN-2013) Infectious Diseases, SJCRH, 262 Danny
            Thomas Place, Memphis, TN 38105, USA
FEATURES             Location/Qualifiers
     source          1..1722
                     /organism="Influenza A virus
                     (A/duck/Egypt/S6419C/2012(H5N1))"
                     /mol_type="viral cRNA"
               

...

LOCUS       KY171584                1704 bp    cRNA    linear   VRL 07-DEC-2016
DEFINITION  Influenza A virus (A/duck/Vietnam/NCVD-0033/2012(H5N1)) segment 4
            hemagglutinin (HA) gene, complete cds.
ACCESSION   KY171584
VERSION     KY171584.1
KEYWORDS    .
SOURCE      Influenza A virus (A/duck/Vietnam/NCVD-0033/2012(H5N1))
  ORGANISM  Influenza A virus (A/duck/Vietnam/NCVD-0033/2012(H5N1))
            Viruses; ssRNA viruses; ssRNA negative-strand viruses;
            Orthomyxoviridae; Influenzavirus A.
REFERENCE   1  (bases 1 to 1704)
  AUTHORS   Davis,T. and Jang,Y.
  TITLE     Influenza Sequencing Activity group
  JOURNAL   Unpublished
REFERENCE   2  (bases 1 to 1704)
  AUTHORS   Davis,T. and Jang,Y.
  TITLE     Direct Submission
  JOURNAL   Submitted (16-NOV-2016) WHO Collaborating Center for Surveillance,
            Epidemiology and Control of Influenza, Influenza Division, Centers
            for Disease Control and Prevention, 1600 Clifton Road, N.E.,
            Atlan

LOCUS       MG042249                1748 bp    cRNA    linear   VRL 05-NOV-2017
DEFINITION  Influenza A virus (A/duck/Bangladesh/30885/2016(H5N1)) segment 4
            hemagglutinin (HA) gene, complete cds.
ACCESSION   MG042249
VERSION     MG042249.1
DBLINK      BioProject: PRJNA298519
KEYWORDS    .
SOURCE      Influenza A virus (A/duck/Bangladesh/30885/2016(H5N1))
  ORGANISM  Influenza A virus (A/duck/Bangladesh/30885/2016(H5N1))
            Viruses; ssRNA viruses; ssRNA negative-strand viruses;
            Orthomyxoviridae; Influenzavirus A.
REFERENCE   1  (bases 1 to 1748)
  AUTHORS   Barman,S., Turner,J.C., Hasan,M.K., Akhtar,S., Franks,J.,
            El-Shesheny,R., Walker,D., Seiler,P., Friedman,K., Kercher,L.,
            Kayali,G., Jones-Engel,L., McKenzie,P., Krauss,S., Webby,R.J.,
            Feeroz,M.M. and Webster,R.G.
  CONSRTM   Centers of Excellence for Influenza Research and Surveillance
            (CEIRS)
  TITLE     Direct Submission
  JOURNAL   Submitted (04-OCT-2

LOCUS       LC167710                1757 bp    cRNA    linear   VRL 26-JUL-2016
DEFINITION  Influenza A virus (A/duck/Vietnam/HU3-1126/2015(H5N1)) viral cRNA,
            segment 4, complete sequence.
ACCESSION   LC167710
VERSION     LC167710.1
KEYWORDS    .
SOURCE      Influenza A virus (A/duck/Vietnam/HU3-1126/2015(H5N1))
  ORGANISM  Influenza A virus (A/duck/Vietnam/HU3-1126/2015(H5N1))
            Viruses; ssRNA viruses; ssRNA negative-strand viruses;
            Orthomyxoviridae; Influenzavirus A.
REFERENCE   1
  AUTHORS   Nguyen,T.L., Hiono,T., Matsuno,K., Okamatsu,M., Kida,H. and
            Sakoda,Y.
  TITLE     Characterization of avian influenza viruses isolated from poultry
            in Vietnam in 2015
  JOURNAL   Unpublished
REFERENCE   2  (bases 1 to 1757)
  AUTHORS   Hatamachi,J., Kida,H., Sakoda,Y., Okamatsu,M., Matsuno,K. and
            Hiono,T.
  TITLE     Direct Submission
  JOURNAL   Submitted (09-JUL-2016) Contact:Jun Hatamachi Hokkaido University,
            Gr

LOCUS       KX513156                1776 bp    cRNA    linear   VRL 29-JUL-2016
DEFINITION  Influenza A virus (A/duck/Vietnam/QT1484/2012(H5N1)) segment 4
            hemagglutinin (HA) gene, complete cds.
ACCESSION   KX513156
VERSION     KX513156.1
DBLINK      BioProject: PRJNA323813
KEYWORDS    .
SOURCE      Influenza A virus (A/duck/Vietnam/QT1484/2012(H5N1))
  ORGANISM  Influenza A virus (A/duck/Vietnam/QT1484/2012(H5N1))
            Viruses; ssRNA viruses; ssRNA negative-strand viruses;
            Orthomyxoviridae; Influenzavirus A.
REFERENCE   1  (bases 1 to 1776)
  CONSRTM   Centers of Excellence for Influenza Research and Surveillance
            (CEIRS)
  TITLE     Direct Submission
  JOURNAL   Submitted (06-JUL-2016) Center for Research on Influenza
            Pathogenesis (CRIP), New York, NY 10029-6574, USA
COMMENT     ##Assembly-Data-START##
            Assembly Method       :: Trinity/Inchworm v. r
            Sequencing Technology :: Illumina
            ##Assembly-Dat

Supplied id parameter is empty.


Supplied id parameter is empty.


Supplied id parameter is empty.


Supplied id parameter is empty.


Supplied id parameter is empty.


Supplied id parameter is empty.


Supplied id parameter is empty.


Supplied id parameter is empty.


Supplied id parameter is empty.


Supplied id parameter is empty.


Supplied id parameter is empty.


Supplied id parameter is empty.


Supplied id parameter is empty.


Supplied id parameter is empty.


Supplied id parameter is empty.


Supplied id parameter is empty.



In [17]:
print(extra_PMIDs)

[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]


### Results: 

Ok, this makes sense. Apparently, all of the genbank entries that are not in my list are ones that do not have associated PUBMED IDs. However, it looks like for many of them, you can figure out whether they are domestic or wild based on the title of the submission. I will do that manually now. The results are saved in 

In [18]:
# add those new genbank accessions to PMIDs2

print(len(PMIDs2))
genbankids2 = "/Users/lmoncla/Documents/Fellowships/data_query/domestic_wild_annotations_genbankaccession_2018-08-05.txt"

with open(genbankids2, "r") as infile:
    for line in infile:
        genbankid = line.split("\t")[0]
        domestic_wild = line.strip().split("\t")[1]
        
        PMIDs2[genbankid] = domestic_wild
        
print(PMIDs2)

6496
{'jx497773': 'wild', 'jx497774': 'wild', 'jx497775': 'wild', 'jx497776': 'wild', 'jx497777': 'wild', 'jx497778': 'wild', 'jx497779': 'wild', 'jx497772': 'wild', 'jx497780': 'wild', 'jx497765': 'wild', 'jx497766': 'wild', 'jx497767': 'wild', 'jx497768': 'wild', 'jx497769': 'wild', 'jx497770': 'wild', 'jx497771': 'wild', 'eu057725': 'wild', 'jf510040': 'wild', 'jf510045': 'wild', 'jf510046': 'wild', 'jf510041': 'wild', 'jf510043': 'wild', 'jf510047': 'wild', 'jf510042': 'wild', 'jf510044': 'wild', 'kp739418': 'wild', 'kp739419': 'wild', 'kp739420': 'wild', 'kp739421': 'wild', 'kp739422': 'wild', 'kp739423': 'wild', 'kp739424': 'wild', 'kp739425': 'wild', 'kc690153': 'domestic', 'kc690154': 'domestic', 'kc690155': 'domestic', 'kc690156': 'domestic', 'kc690157': 'domestic', 'kc690158': 'domestic', 'kc690159': 'domestic', 'kc690160': 'domestic', 'ab780475': 'domestic', 'ab780476': 'domestic', 'ab780477': 'domestic', 'ab780478': 'domestic', 'ab780479': 'domestic', 'ab780480': 'domestic'

In [19]:
# now check again

alignment = "/Users/lmoncla/Documents/H5N1_Cambodian_outbreak_study/beast/dta-2018-07-05/subsampling/HA_2018-07-19.aligned.reordered.fasta"

yes = 0 
no = 0
not_available = 0

with open(alignment, "r") as infile: 
    for line in infile:
        if ">" in line: 
            strain = line.split("|")[0]
            if len(strain.split("/")) == 4:
                species = "human"
            elif len(strain.split("/")) < 4:
                species = "unknown"
            else:
                species = strain.split("/")[1]
                
            genbank = line.split("|")[6]
            
            if species == "duck" or species == "goose":
                if genbank in PMIDs2:
                    #print("yes")
                    yes+= 1
                else:
                    if genbank == 'not_available' or genbank == 'unknown':
                    #print("no", genbank)
                        not_available += 1
                    
                    else:
                        no += 1
                        print(genbank)
                        no_list.append(genbank)
                    
print(yes, no, not_available)

174 0 132


## Match up sequence names to their domestic wild status

In [20]:
# classify into wild and domestic lists

wild_list = ['waterfowl', 'greyheron', 'blackcrownednightheron', 'grebe', 'maraboustork', 'wildduck', 'eurasianeagleowl', 'quail', 'hillmyna', 'gadwallduck', 'peregrinefalcon', 'commongoldeneye', 'americangreenwingedteal', 'wildbird', 'whooperswan', 'hoodedcrane', 'muteswan', 'blackswan', 'mandarinduck', 'greategret', 'barnswallow', 'brownheadedgull', 'turtledove', 'swallow', 'ruddyshelduck', 'greatcrestedgrebe', 'beangoose', 'northernshoveler', 'ngan', 'greatblackheadedgull', 'blackduck', 'barheadedgoose', 'dove', 'canadagoose', 'partridge', 'swan', 'rubbyshelduck', 'magpierobin', 'ruddyturnstone', 'pigeon', 'japanesewhiteeye', 'tuftedduck', 'pheasant', 'swiftlet', 'crow', 'americanwigeon']
domestic_list = ['domesticduck','domesticbird','poultry','chicken']

In [21]:
# read in annotations by strain name text file and make 1 more dictionary with the strain name : domestic/wild status

annotations_by_strain_name = "/Users/lmoncla/Documents/Fellowships/data_query/annotations_by_strain_name_2018-06-29.txt"

strain_name_annotations = {}

with open(annotations_by_strain_name, "r") as infile: 
    for line in infile: 
        strain = line.split("\t")[1].lower()
        domestic_wild = line.split("\t")[2].strip().lower()
        
        strain_name_annotations[strain] = domestic_wild

In [22]:
alignment = "/Users/lmoncla/Documents/H5N1_Cambodian_outbreak_study/beast/dta-2018-07-05/subsampling/HA_2018-07-19.aligned.reordered.fasta"
outfilename = "/Users/lmoncla/Documents/Fellowships/data_query/H5N1_tree_annotations.txt"

unique_species = []
human = 0
wild = 0
domestic = 0
unknown = 0
other = 0

with open(alignment, "r") as infile: 
    for line in infile:
        if ">" in line: 
            strain = line.split("|")[0].replace(">","")
            genbank = line.split("|")[6]
            
            if len(strain.split("/")) == 4:
                species = "human"
                host_group = "human"
            
            elif len(strain.split("/")) < 4:
                species = "unknown"
                host_group = "unknown"
                
            else:
                species = strain.split("/")[1]
                
                if species in wild_list: 
                    host_group = "wild"
            
                elif species in domestic_list:
                    host_group = "domestic"
            
                elif genbank.lower() in PMIDs2:
                    host_group = PMIDs2[genbank].lower()
                    
                elif strain.lower() in strain_name_annotations:
                    host_group = strain_name_annotations[strain].lower()
                    
                else:
                    host_group = "unknown" 
            
            
            if host_group == "human":
                human += 1
            
            elif host_group == "wild":
                wild += 1
                
            elif host_group == "domestic":
                domestic += 1
                
            elif host_group == "unknown" or host_group == "":
                unknown += 1
                
            else:
                other += 1
                
            annotations = [line.replace(">","").strip(), strain, species, host_group]
            annotations = "\t".join(annotations)
            with open(outfilename, "a") as outfile: 
                outfile.write(annotations + "\n")

print(human, wild, domestic, unknown, other)

68 102 388 361 2
